# Packages used

In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt

import os
import glob
import numpy as np

## Link to data 
- combined_ALL_31points_mean.fits **[size: ~ 4 GB]**
    - https://drive.google.com/file/d/1zQtdioDBiiyipUPHV2MEZi6j_cKJSfSH/view?usp=drive_link

In [2]:
caminho_do_arquivo = "combined_ALL_31points_mean.fits"

In [3]:
hdul = fits.open(caminho_do_arquivo)  
data_stokes = hdul[0].data

data_stokes.shape

(4395504, 124)

# Normalized by maximum intensity

In [4]:
stokes = np.copy(data_stokes)
stokes = stokes.astype("float64")  

I_qs = np.max(stokes)
stokes /= I_qs

# Random selection of 10% of the total data

In [5]:
num_linhas_total = stokes.shape[0]
num_linhas_selecionar = int(num_linhas_total * 0.10)

# Generate random indexes for rows, 'replace=False' ensures you don't select the same row more than once.
indices_selecionados = np.random.choice(num_linhas_total, num_linhas_selecionar, replace=False)

# Use indexes to select rows from your array
data_for_som = stokes[indices_selecionados]

# SOM

- ### Packages used for SOM
  - **MiniSom :**
     'https://github.com/JustGlowing/minisom

In [6]:
from minisom import MiniSom

In [7]:
np.random.seed(24)
Nnumber  = num_linhas_selecionar
data_SOM = data_for_som

# SOM Parameters
function = 'gaussian'  
topology = 'hexagonal'
metric   = 'euclidean'

tamanho_mapa = (5, 5)    # Grid size 2D
sigma = 2                # Initial neighborhood radius
taxa_aprendizado = 1.5   # Learning rate
n_iteracoes = Nnumber    # Number of iterations 

### Initialize and train SOM 

In [8]:
som = MiniSom(tamanho_mapa[0], tamanho_mapa[1], data_SOM.shape[1], 
              sigma=sigma,
              topology = topology, 
              neighborhood_function = function,
              activation_distance = metric,
              learning_rate=taxa_aprendizado, 
              random_seed=24)

som.train_random(data_SOM, n_iteracoes, verbose=True)

 [ 439550 / 439550 ] 100% - 0:00:00 left 
 quantization error: 0.18374190201784824


###  Quantization and topographic errors

In [9]:
qe = som.quantization_error(data_SOM)
te = som.topographic_error(data_SOM)

qe, te

(np.float64(0.18374190201784824), np.float64(0.10439540439085426))

### Save

In [10]:
# Save weights
np.save('Weights_SOM_31points_Imax.npy', som.get_weights())

# Save parameters
params = {
    'shape': som.get_weights().shape,
    'sigma': sigma,
    'learning_rate': taxa_aprendizado,
    'topology':topology, 
    'neighborhood_function':function
}
with open('PARAMS_SOM_31points_Imax.txt', 'w') as f:
    f.write(str(params))